# Upskills clustering

A number of experiments on trying to make sense of the ~200 job offers compiled as part of the upskills project. 
The core idea is trying to organise the kib offers. Since we do not have any annotation, we have opted for carrying out a clustering. Both representations and clustering come from [scikit](https://scikit-learn.org/) (twiking a bit the tokenizer)

**Representations**


* [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
* [SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) (LSA)

I had considered doc2vec at first, but the computation was taking way too long

**Clustering alternatives**
* [Birch](https://scikit-learn.org/stable/modules/clustering.html#birch). Requires the number of clusters. 
* [Afifinity propagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation). Estimates the right number of clusters.
* [Meanshift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift).  Estimates the right number of clusters.

**Requirements (non-python-standard)**
* spacy 3.0. Tokenization, lemmatization
* sklearn. Feature computation, clustering
* pandas. Dataframes 

In [ ]:
# Downloading the model for spacy 
# RUN ONLY IF YOU DON'T HAVE THE MODELS READY
! python3 -m spacy download en_core_web_sm
# ! python -V
# ! which python
# ! pip3 install --upgrade --upgrade-strategy eager sklearn
# ! pip3 install --upgrade spacy

In [ ]:
import spacy as kk
  
# Check the version 
print(kk.__version__) 

In [ ]:
# import gensim.models as g
import os
import pandas as pd
import xml.etree.ElementTree as ET

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from sklearn.decomposition import PCA, TruncatedSVD
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import AffinityPropagation
from sklearn.cluster import Birch
from sklearn.cluster import MeanShift

from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.preprocessing import MinMaxScaler

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

In [ ]:
# YOU SHOULD CHANGE THIS PATH TO YOUR LOCAL SETTINGS
path = "/Users/albarron/tmp/adriano/txt_raw"

# From when considering to use doc2vec. Not really necessary
# model = "/Users/albarron/corpora/embeddings/doc2vec/enwiki_dbow/doc2vec.bin"
#load model
# m = g.Doc2Vec.load(model)

In [ ]:
def find_files(path):
    """Loads all the txt files in the path folder and 
    returns them, with their full path"""
    my_files = []
    for root, dirs, files in os.walk(path):
        my_files.extend(os.path.join(path, file) for file in files if file.endswith(".txt"))
    return my_files

In [ ]:
def extract_xml(file):
    """Extract the contents from an xml file (which in this 
    corpus actually have txt extension) and returns it as 
    a dictionary. 
    The assumed tags are: 'id', 'jobtitle', 'about', 'jobdesc', 
    'keyinfo', 'benefits'.
    If jobdesc contains internal tags, all their text is simply 
    merged.
    """
    with open(file) as f:
        elements = {}
        tree = ET.fromstring(f.read())
        elements['id'] = tree.attrib['id']

        for child in tree: #ET.fromstring(f.read()):
            # elements[child.tag] = child.text
            elements[child.tag] = ' '.join(child.itertext()).lower()
    return elements

## Load all the files into a pandas dataframe

In [ ]:
files = find_files(path)
df = pd.DataFrame(
    columns=['id', 'jobtitle', 'about', 'jobdesc', 'keyinfo', 'benefits'])

for file in files:
#     print(file)
    d = extract_xml(file)
    df = df.append(d, ignore_index=True)

# replace empty jobtitles (NAN in a dataframe) with ''
df.jobtitle = df.jobtitle.fillna('')
# Add a new column combining offer title and description
df['jobtitle_desc'] = " " + df.jobtitle + '\n' + df.jobdesc
df.set_index('id')
# print(df.jobdesc)
print(df.jobtitle_desc)

## Representations production

### Alternative 1: tf-idf

In [ ]:
doc = nlp("this is a very small tests trying to check if I am or you are actually getting lemmas and not tokens")
lemmas = set([w.lemma for w in doc])
tokens = set([w for w in doc])
print (len(lemmas), len(tokens))
print(doc)
nlp.add_pipe("lemmatizer", config={"mode": "lookup"})
# config = {"mode": "rule"}
# nlp.add_pipe("lemmatizer", config=config)
# This usually happens under the hood
processed = nlp(doc)


In [ ]:
# tfidf_docs = pd.DataFrame(tfidf_docs)
# # # centers the vectorized documents (BOW vectors) by subtracting the mean
# # tfidf_docs = tfidf_docs - tfidf_docs.mean()
# import spacy


# stemmer = FrenchStemmer()
# analyzer = CountVectorizer().build_analyzer()

def lemmatized_words(doc):
    """A tokenizer based on spacy to add lemmas to the vector, 
    rather than tokens. It also ignores sequences of spaces 
    and 1-character tokens"""
    doc = nlp(doc)
    return (w.lemma_ for w in doc if w.lemma_.strip() !="" and len(w.lemma_) > 1)

# Without lemmatization; the vocabulary is huge: 75.4k.
# lemm_vectorizer = TfidfVectorizer(tokenizer=tokenizer) 

# Without lemmatization; the vocabulary is reasonable < 3k
lemm_vectorizer = TfidfVectorizer(
    tokenizer=lemmatized_words, # the tokenizer from the function 
    stop_words='english', 
    min_df=2)

tfidf_docs = lemm_vectorizer.fit_transform(raw_documents=df.jobtitle_desc).toarray()
print("Size of the vocabulary:", len(lemm_vectorizer.vocabulary_))

tfidf_docs = pd.DataFrame(tfidf_docs)
print("Shape of the matrix:", tfidf_docs.shape)

# Uncomment if you want to see the features (vocabulary)
print("Feature names:\n", lemm_vectorizer.get_feature_names())

# print(tfidf_docs)

## Alternative 2: PCA

In [ ]:
scaler = MinMaxScaler()
tfidf_docs_rescaled = scaler.fit_transform(tfidf_docs)

pca = PCA(svd_solver = 'full', n_components=20)
pca = pca.fit(tfidf_docs_rescaled)
pca_topic_vectors = pca.transform(tfidf_docs_rescaled)

columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=df.id)
pca_topic_vectors.round(3).head(6)

## Alternative 3: TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=100, n_iter=100)
scaler = MinMaxScaler()
tfidf_docs_rescaled = scaler.fit_transform(tfidf_docs)

# Decomposes TF-IDF vectors and transforms them into topic vectors
svd_topic_vectors = svd.fit(tfidf_docs_rescaled)
svd_topic_vectors = svd.transform(tfidf_docs_rescaled)
# print(svd_topic_vectors.shape)
columns = ['topic{}'.format(i) for i in range(svd.n_components)]
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=df.id)
# Same as those produced by PCA
svd_topic_vectors.round(3).head(6)

# Clustering

## Alternative 1: Birch (which requires the number of clusters)

In [ ]:
for k in range(1,21):    
    brc = Birch(branching_factor=50, n_clusters=k, threshold=0.1, compute_labels=True)
    brc.fit(tfidf_docs)
     
    clusters = brc.predict(tfidf_docs)
     
    labels = brc.labels_
     
    df[str(k)] = clusters 
    # print ("Clusters: ")
    # print (clusters)


In [ ]:
print(df.head(10))
df.to_csv("/".join([path, "upskills_clusters.tsv"]), sep="\t")

# Trying with [AffinityPropagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation) with tf-idf vectors

In [ ]:
# #############################################################################
# Compute Affinity Propagation
X = tfidf_docs
# centers the vectorized documents (BOW vectors) by subtracting the mean
X = X - X.mean()

af = AffinityPropagation(random_state=None)
af.fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
print(labels)

# Trying with [AffinityPropagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation) with PCA vectors

In [ ]:
# #############################################################################
# Compute Affinity Propagation
X = pca_topic_vectors
af = AffinityPropagation()
af.fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
print(labels)

# Trying with [AffinityPropagation](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation) with SVD (LSA) vectors

In [ ]:
# #############################################################################
# Compute Affinity Propagation
X = svd_topic_vectors
af = AffinityPropagation()
af.fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels, metric='sqeuclidean'))
print(labels)

for id, cluster in zip(df.id, labels):
    print (id, cluster)
# print("\t".join([id, cluster]) for id, cluster in zip (df.id, labels))

# Clustering with Meanshift

In [ ]:
cluster_centers_indices

In [ ]:

 # Trying with doc2vec 
 # The computation is way to slow and we might need to use the cluster
 
# test_docs="data/test_docs.txt"
 


# load the texts into lists
test_ids = []
test_docs = []
X = []
for i, row in df.iterrows():
  print(row['id'])
  test_ids.append(row['id'])
  X.append(
      m.infer_vector(
          row['jobdesc'].strip().split(),
          # [x.strip().split() for x in row['jobdesc']], 
          alpha=start_alpha, steps=infer_epoch) )
      

print(X)


# test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r", "utf-8").readlines() ]
 
# print (test_docs)
# """
# [['the', 'cardigan', 'welsh', 'corgi'........
# """
 
# X=[]
# for d in test_docs:
     
#     X.append( m.infer_vector(d, alpha=start_alpha, steps=infer_epoch) )
    


In [ ]:

 
k=3
 
 
brc = Birch(branching_factor=50, n_clusters=k, threshold=0.1, compute_labels=True)
brc.fit(X)
 
clusters = brc.predict(X)
 
labels = brc.labels_
 
 
print ("Clusters: ")
print (clusters)
 
 
# silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
# print ("Silhouette_score: ")
# print (silhouette_score)


The community has created multiple libraries for pre-processing, which include options for tokenisation. One of the most popular ones is [NLTK](http://www.nltk.org). 

Before using it, you should install it. If using pip, you should do: 

\$ pip install --user -U nltk

\$ pip install --user -U numpy


An now we can import and use one of its tokenisers

In [ ]:
from nltk.tokenize import TreebankWordTokenizer # import one of the many tokenizers available
tokenizer = TreebankWordTokenizer()             # invoke it 
tokens = tokenizer.tokenize(txt)
print(tokens)

Now, see the difference between tokenising with split() and with NLTK's treebank tokeniser on a different sentence.

In [ ]:
sentence = "Monticello wasn't designated as UNESCO World Heritage Site until 1987."
tokens_split = sentence.split()
tokens_tree = tokenizer.tokenize(sentence)

print("OUTPUT USING split()\t\t", tokens_split)
print("OUTPUT USING TreebankWordTokenizer\t", tokens_tree)

## Normalisation

### Casefolding

In [ ]:
sentence  = sentence.lower()
print(sentence)

## Stemming

Once again, we can use a regular expression to do stemming

In [ ]:
def stem(phrase):
    return ' '.join([re.findall('^(.*ss|.*?)(s)?$',
         word)[0][0].strip("'") for word in phrase.lower()
         .split()])

In [ ]:
print("'houses' \t\t->", stem('houses'))
print("'Doctor House's calls' \t->", stem("Doctor House's calls"))
print("'stress' \t\t->", stem("stress"))

But we would need to include many more expressions to deal with all cases and exceptions.

Instead, once again we can rely on a library. Let's consider the **Porter stemmer**, available in NLTK.

In [ ]:
from nltk.stem.porter import PorterStemmer # Import the stemmer
stemmer = PorterStemmer()                  # invoke the stemmer

# Notice that we are "tokenising" and stemming in one line
x = ' '.join([stemmer.stem(w).strip("'") for w in "dish washer's washed dishes".split()])
print(x.split())

## Lemmatisation

This is a more complex process, compared to stemming. Let us go straight to use a library.
In this particular case we are going to use NLTK's WordNet lemmatiser. If it is the first time you use it (or you are in an ephemeral environment!), you should download it as follows:

In [ ]:
import nltk 
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer # importing the lemmatiser
lemmatizer = WordNetLemmatizer()        # invoking it

print("'better' alone \t->",lemmatizer.lemmatize("better"))
print("'better' including it's part of speech (adj) \t->",lemmatizer.lemmatize("better", pos="a"))

## A quick overview on representations

### Bag of Words (BoW)

First, let us see a simple construction, using a dictionary

In [ ]:
sentence = """Thomas Jefferson began building Monticello at the age of 26. Thomas"""

sentence_bow = {}
for token in sentence.split():
     sentence_bow[token] = 1
sorted(sentence_bow.items())


Another option would be using **pandas**

In [ ]:
import pandas as pd

# Loading the corpus
sentences = """Thomas Jefferson began building Monticello at the age of 26.\n"""
sentences += """Construction was done mostly by local masons and carpenters.\n"""
sentences += "He moved into the South Pavilion in 1770.\n"
sentences += """Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."""

# Loading the tokens into a dictionary (notice that we asume that each line is a document)
corpus = {}
for i, sent in enumerate(sentences.split('\n')):
    corpus['sent{}'.format(i)] = dict((tok, 1) for tok in
         sent.split())

# Loading the dictionary contents into a pandas dataframe. 
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
# SEE THE .T, which transposes the matrix for visualisation purposes.


df[df.columns[:10]]


### One-hot vectors

This is our input sentence (and its vocabulary)

In [ ]:
import numpy as np
sentence = "Thomas Jefferson began building Monticello at the age of 26."
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))
print(vocab)

And now, we produce the one-hot representation

In [ ]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)
onehot_vectors = np.zeros((num_tokens, vocab_size), int) # create the |tokens| x |vocabulary size| matrix of zeros 
for i, word in enumerate(token_sequence):
   onehot_vectors[i, vocab.index(word)] = 1  # set one to right dimension to 1

print("Vocabulary:\t", vocab)
print("Sentence:\t", token_sequence)
onehot_vectors

Let us bring pandas into the game

In [ ]:
pd.DataFrame(onehot_vectors, columns=vocab)